In [1]:
"""
Führt Regressionsanalyse durch

"""
import pandas as pd
import numpy as np



#Initmessage
print ("starting Classifier") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )
print("data collected")
print(data_frame_investigated.head())

starting Classifier
data collected
      slug symbol     name        date  ranknow    open    high     low  \
0  bitcoin    BTC  Bitcoin  2013-04-28        1  135.30  135.98  132.10   
1  bitcoin    BTC  Bitcoin  2013-04-29        1  134.44  147.49  134.00   
2  bitcoin    BTC  Bitcoin  2013-04-30        1  144.00  146.93  134.05   
3  bitcoin    BTC  Bitcoin  2013-05-01        1  139.00  139.89  107.72   
4  bitcoin    BTC  Bitcoin  2013-05-02        1  116.38  125.60   92.28   

    close  volume        market  close_ratio  spread  
0  134.21     0.0  1.500520e+09       0.5438    3.88  
1  144.54     0.0  1.491160e+09       0.7813   13.49  
2  139.00     0.0  1.597780e+09       0.3843   12.88  
3  116.99     0.0  1.542820e+09       0.2882   32.17  
4  105.21     0.0  1.292190e+09       0.3881   33.32  


In [2]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe
print(data_frame_investigated.head())

      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  
0  1.500520e+09       0.5438    3.88  
1  1.491160e+09       0.7813   13.49  
2  1.597780e+09       0.3843   12.88  
3  1.542820e+09       0.2882   32.17  
4  1.292190e+09       0.3881   33.32  


In [104]:
data_frame_investigated['close_2'] = data_frame_investigated['close']


df = data_frame_investigated['close_2'].shift(periods=1).to_frame()
print(data_frame_investigated)


#df_result = pd.merge(data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin'], df_t, on=['TMP'])



#df = pd.DataFrame(data=df, columns=['close_2'])

#df_result = pd.merge(data_frame_investigated, df, right_index = True)

#data_frame_investigated['close_2'].shift(periods=1, axis=1)



#print(df_res.head())

KeyError: 'close'

In [81]:
data_frame_investigated = data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin']
#dt_investigated eine spalte namens trend mit 0 hinzufügen
data_frame_investigated['Trend'] = 5

#Spalte close und Trend rausschneiden
df_cl = data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin']['close'] 
df_t  = data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin']['Trend']
#df_t = df_t[df_t!=5]


z = 0
z2 = 0
#vergleiche mit nachfolgern in spalte close und bearbeite trend entsrpechend
for index in df_cl.index.tolist():
    z2 +=1
    print(" index:",index)
    if (df_cl[index] != max(df_cl.index.tolist()) and df_cl[index] != max(df_cl.index.tolist())-1):
        
        if df_cl[index] < df_cl[index+1]:
            print(index)
            print("eins: ", "c1: ", df_cl[index], "c2: ", df_cl[index+1] )
            df_t = df_t.set_value(index, 1)  #  heißt kurs ist danach gestiegen
            z +=1
        else:
            print(index)
            print("nulll: ", "c1: ", df_cl[index], "c2: ", df_cl[index+1] )
            df_t = df_t.set_value(index, 0) # 0 heißt kurs ist danach gefallen oder gleich
            z+=1



print("wert setzen: ", z)
print("for schleife: ", z2)
print("max index ", max(df_cl.index.tolist()))
df_t = pd.DataFrame(data=df_t, columns=['Trend'])
print (df_cl.tail(30))

 index schleife: 0
0
eins:  c1:  134.21 c2:  144.54
 index schleife: 1
1
nulll:  c1:  144.54 c2:  139.0
 index schleife: 2
2
nulll:  c1:  139.0 c2:  116.99
 index schleife: 3
3
nulll:  c1:  116.99 c2:  105.21
 index schleife: 4
4
nulll:  c1:  105.21 c2:  97.75
 index schleife: 5
5
eins:  c1:  97.75 c2:  112.5
 index schleife: 6
6
eins:  c1:  112.5 c2:  115.91
 index schleife: 7
7
nulll:  c1:  115.91 c2:  112.3
 index schleife: 8
8
nulll:  c1:  112.3 c2:  111.5
 index schleife: 9
9
eins:  c1:  111.5 c2:  113.57
 index schleife: 10
10
nulll:  c1:  113.57 c2:  112.67
 index schleife: 11
11
eins:  c1:  112.67 c2:  117.2
 index schleife: 12
12
nulll:  c1:  117.2 c2:  115.24
 index schleife: 13
13
nulll:  c1:  115.24 c2:  115.0
 index schleife: 14
14
eins:  c1:  115.0 c2:  117.98
 index schleife: 15
15
nulll:  c1:  117.98 c2:  111.5
 index schleife: 16
16
eins:  c1:  111.5 c2:  114.22
 index schleife: 17
17
eins:  c1:  114.22 c2:  118.76
 index schleife: 18
18
eins:  c1:  118.76 c2:  123.02


/Users/Jonas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/Jonas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



eins:  c1:  126.0 c2:  130.69
 index schleife: 164
164
nulll:  c1:  130.69 c2:  130.59
 index schleife: 165
165
eins:  c1:  130.59 c2:  130.9
 index schleife: 166
166
eins:  c1:  130.9 c2:  135.19
 index schleife: 167
167
eins:  c1:  135.19 c2:  138.13
 index schleife: 168
168
eins:  c1:  138.13 c2:  140.52
 index schleife: 169
169
eins:  c1:  140.52 c2:  145.24
 index schleife: 170
170
nulll:  c1:  145.24 c2:  142.55
 index schleife: 171
171
eins:  c1:  142.55 c2:  146.25
 index schleife: 172
172
eins:  c1:  146.25 c2:  155.96
 index schleife: 173
173
eins:  c1:  155.96 c2:  172.42
 index schleife: 174
174
eins:  c1:  172.42 c2:  174.61
 index schleife: 175
175
eins:  c1:  174.61 c2:  182.21
 index schleife: 176
176
eins:  c1:  182.21 c2:  193.76
 index schleife: 177
177
eins:  c1:  193.76 c2:  213.62
 index schleife: 178
178
nulll:  c1:  213.62 c2:  198.23
 index schleife: 179
179
nulll:  c1:  198.23 c2:  186.69
 index schleife: 180
180
nulll:  c1:  186.69 c2:  177.32
 index schleif

nulll:  c1:  441.46 c2:  440.67
 index schleife: 380
380
eins:  c1:  440.67 c2:  443.97
 index schleife: 381
381
eins:  c1:  443.97 c2:  447.25
 index schleife: 382
382
eins:  c1:  447.25 c2:  448.06
 index schleife: 383
383
eins:  c1:  448.06 c2:  448.9
 index schleife: 384
384
nulll:  c1:  448.9 c2:  446.26
 index schleife: 385
385
nulll:  c1:  446.26 c2:  446.18
 index schleife: 386
386
eins:  c1:  446.18 c2:  485.72
 index schleife: 387
387
eins:  c1:  485.72 c2:  491.77
 index schleife: 388
388
eins:  c1:  491.77 c2:  524.58
 index schleife: 389
389
nulll:  c1:  524.58 c2:  520.22
 index schleife: 390
390
eins:  c1:  520.22 c2:  525.14
 index schleife: 391
391
eins:  c1:  525.14 c2:  571.59
 index schleife: 392
392
eins:  c1:  571.59 c2:  583.42
 index schleife: 393
393
nulll:  c1:  583.42 c2:  571.24
 index schleife: 394
394
eins:  c1:  571.24 c2:  577.06
 index schleife: 395
395
nulll:  c1:  577.06 c2:  568.18
 index schleife: 396
396
eins:  c1:  568.18 c2:  615.33
 index schlei

 index schleife: 578
578
eins:  c1:  369.67 c2:  376.45
 index schleife: 579
579
nulll:  c1:  376.45 c2:  375.49
 index schleife: 580
580
eins:  c1:  375.49 c2:  378.05
 index schleife: 581
581
eins:  c1:  378.05 c2:  379.25
 index schleife: 582
582
eins:  c1:  379.25 c2:  381.31
 index schleife: 583
583
nulll:  c1:  381.31 c2:  375.01
 index schleife: 584
584
nulll:  c1:  375.01 c2:  369.6
 index schleife: 585
585
eins:  c1:  369.6 c2:  376.85
 index schleife: 586
586
nulll:  c1:  376.85 c2:  374.79
 index schleife: 587
587
eins:  c1:  374.79 c2:  375.1
 index schleife: 588
588
nulll:  c1:  375.1 c2:  361.91
 index schleife: 589
589
nulll:  c1:  361.91 c2:  352.22
 index schleife: 590
590
nulll:  c1:  352.22 c2:  346.37
 index schleife: 591
591
eins:  c1:  346.37 c2:  350.51
 index schleife: 592
592
eins:  c1:  350.51 c2:  352.54
 index schleife: 593
593
nulll:  c1:  352.54 c2:  347.38
 index schleife: 594
594
eins:  c1:  347.38 c2:  351.63
 index schleife: 595
595
nulll:  c1:  351.63

...

eins:  c1:  610.89 c2:  612.13
 index schleife: 1254
1254
nulll:  c1:  612.13 c2:  610.2
 index schleife: 1255
1255
eins:  c1:  610.2 c2:  612.51
 index schleife: 1256
1256
eins:  c1:  612.51 c2:  613.02
 index schleife: 1257
1257
eins:  c1:  613.02 c2:  617.12
 index schleife: 1258
1258
eins:  c1:  617.12 c2:  619.11
 index schleife: 1259
1259
nulll:  c1:  619.11 c2:  616.75
 index schleife: 1260
1260
eins:  c1:  616.75 c2:  618.99
 index schleife: 1261
1261
eins:  c1:  618.99 c2:  641.07
 index schleife: 1262
1262
nulll:  c1:  641.07 c2:  636.19
 index schleife: 1263
1263
eins:  c1:  636.19 c2:  636.79
 index schleife: 1264
1264
eins:  c1:  636.79 c2:  640.38
 index schleife: 1265
1265
nulll:  c1:  640.38 c2:  638.65
 index schleife: 1266
1266
eins:  c1:  638.65 c2:  641.63
 index schleife: 1267
1267
nulll:  c1:  641.63 c2:  639.19
 index schleife: 1268
1268
nulll:  c1:  639.19 c2:  637.96
 index schleife: 1269
1269
nulll:  c1:  637.96 c2:  630.52
 index schleife: 1270
1270
eins:  c1

eins:  c1:  1182.94 c2:  1193.91
 index schleife: 1450
1450
eins:  c1:  1193.91 c2:  1211.67
 index schleife: 1451
1451
nulll:  c1:  1211.67 c2:  1210.29
 index schleife: 1452
1452
eins:  c1:  1210.29 c2:  1229.08
 index schleife: 1453
1453
nulll:  c1:  1229.08 c2:  1222.05
 index schleife: 1454
1454
eins:  c1:  1222.05 c2:  1231.71
 index schleife: 1455
1455
nulll:  c1:  1231.71 c2:  1207.21
 index schleife: 1456
1456
eins:  c1:  1207.21 c2:  1250.15
 index schleife: 1457
1457
eins:  c1:  1250.15 c2:  1265.49
 index schleife: 1458
1458
eins:  c1:  1265.49 c2:  1281.08
 index schleife: 1459
1459
eins:  c1:  1281.08 c2:  1317.73
 index schleife: 1460
1460
nulll:  c1:  1317.73 c2:  1316.48
 index schleife: 1461
1461
eins:  c1:  1316.48 c2:  1321.79
 index schleife: 1462
1462
eins:  c1:  1321.79 c2:  1347.89
 index schleife: 1463
1463
eins:  c1:  1347.89 c2:  1421.6
 index schleife: 1464
1464
eins:  c1:  1421.6 c2:  1452.82
 index schleife: 1465
1465
eins:  c1:  1452.82 c2:  1490.09
 inde

1607
nulll:  c1:  3631.04 c2:  3630.7
 index schleife: 1608
1608
eins:  c1:  3630.7 c2:  3792.4
 index schleife: 1609
1609
nulll:  c1:  3792.4 c2:  3682.84
 index schleife: 1610
1610
eins:  c1:  3682.84 c2:  3926.07
 index schleife: 1611
1611
nulll:  c1:  3926.07 c2:  3892.35
 index schleife: 1612
1612
eins:  c1:  3892.35 c2:  4200.67
 index schleife: 1613
1613
nulll:  c1:  4200.67 c2:  4174.73
 index schleife: 1614
1614
nulll:  c1:  4174.73 c2:  4163.07
 index schleife: 1615
1615
eins:  c1:  4163.07 c2:  4338.71
 index schleife: 1616
1616
eins:  c1:  4338.71 c2:  4403.74
 index schleife: 1617
1617
eins:  c1:  4403.74 c2:  4409.32
 index schleife: 1618
1618
nulll:  c1:  4409.32 c2:  4317.48
 index schleife: 1619
1619
nulll:  c1:  4317.48 c2:  4229.36
 index schleife: 1620
1620
eins:  c1:  4229.36 c2:  4328.41
 index schleife: 1621
1621
eins:  c1:  4328.41 c2:  4370.81
 index schleife: 1622
1622
eins:  c1:  4370.81 c2:  4426.89
 index schleife: 1623
1623
eins:  c1:  4426.89 c2:  4610.48

 index schleife: 1783
1783
nulll:  c1:  8338.35 c2:  7916.88
 index schleife: 1784
1784
eins:  c1:  7916.88 c2:  8223.68
 index schleife: 1785
1785
eins:  c1:  8223.68 c2:  8630.65
 index schleife: 1786
1786
eins:  c1:  8630.65 c2:  8913.47
 index schleife: 1787
1787
eins:  c1:  8913.47 c2:  8929.28
 index schleife: 1788
1788
nulll:  c1:  8929.28 c2:  8728.47
 index schleife: 1789
1789
eins:  c1:  8728.47 c2:  8879.62
 index schleife: 1790
1790
nulll:  c1:  8879.62 c2:  8668.12
 index schleife: 1791
1791
nulll:  c1:  8668.12 c2:  8495.78
 index schleife: 1792


KeyError: 1793

In [41]:
#merge trend mit dt_investigated
data_frame_investigated['TMP'] = 1
df_t['TMP'] = 1

REDUNDANT_ATTRIBUTES=['Trend'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True)

df_result = pd.merge(data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin'], df_t, on=['TMP'])

REDUNDANT_ATTRIBUTES=['TMP'] 
df_result.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True)

print("----df_investigated----", "\n")
print(data_frame_investigated.head(), "\n")

print("----df_trends----", "\n", "\n")
print(df_tt.head())

print("----merge----", "\n")
print(df_result.head())

----df_investigated---- 

      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  TMP  
0  1.500520e+09       0.5438    3.88    1  
1  1.491160e+09       0.7813   13.49    1  
2  1.597780e+09       0.3843   12.88    1  
3  1.542820e+09       0.2882   32.17    1  
4  1.292190e+09       0.3881   33.32    1   

----df_trends---- 
 

   Trend
0      0
1      0
2      0
3      0
4      0
----merge---- 

      slug        date  ranknow   open    high    low   close  volume  \
0  bitcoin  2013-04-28        1  135.3  135.98  132.1  134.21     0.0   
1  bitcoin  2013-04

In [ ]:
#print(df_result['Trend'])
#df_result[df_result['Trend'] == 1
#is_eins = df_result['Trend'] == 1
#print(is_eins)

In [ ]:

from sklearn.model_selection import train_test_split #library fuer Split
from sklearn import linear_model #lineare Regression
import math #import fuer msqError
from sklearn.metrics import mean_squared_error

#dependency :patsy

def cryptoregress(crypto_id, feature_to_compare,crypto_id_feature_name="slug"):
    """
    lineare Regression zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    
   
    print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
        ).reset_index()
    
 
    
    data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    #data_frame_id_exists=data_frame_id_exists.dropna()
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    print("cleaned Frame")
    print(data_frame_id_exists.head())
    
    ###############Train Test Split:
    #Ergebnis vom Features trennen:
    Y = data_frame_id_exists[[crypto_id]]
    data_frame_id_exists.drop(crypto_id,axis=1, inplace = True)
    print("Regress to:")
    print(Y.head())

    
        
    feature_train, feature_test, y_train, y_test = train_test_split (
        data_frame_id_exists,
        Y,
        test_size=0.1
    )

    ##############Regressionsmodell machen
    regression_model = linear_model.LinearRegression(
        fit_intercept=True, normalize=True, copy_X=True, n_jobs=-1)


        #regressionsmodell fitten
    regression_model.fit(feature_train, y_train)
    print("RSquared train:",regression_model.score(feature_train,y_train ))
    print("RSquared test:",regression_model.score(feature_test,y_test ))
    print("MeanSqure train:", math.sqrt( mean_squared_error(regression_model.predict(feature_test),y_test)))
    print("MeanSqure test:", math.sqrt( mean_squared_error(regression_model.predict(feature_train),y_train)))
    
    return regression_model 
       

    


regression_result=cryptoregress('bitcoin','close')

    
    
    

In [ ]:
#zeige mir die Koeffizienten
for cryptoiterator, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_result.coef_[0][cryptoiterator]))

In [ ]:
max(regression_result.coef_)